In [10]:
import numpy as np
import decimal

In [53]:
def get_containing_voxel(bottom_left, top_right, voxel_size, point, direction):
    #find the coordinates of the point on the edges of the voxel space (only works for -1 to 1 range) 
    #then shift in to the appropriate voxel 
    # shift = (np.abs(point) == 1) * ((point < 0) * 2 - 1)*0.5 # don't need? just check if voxel exists in volume
    #coord = coord + shift # not needed
    
    #calculate the steps for the direction
    steps = (direction < 0) * -2 + 1
    steps = steps * (((direction == 0)  & (point >= top_right)) * -2 + 1)

    #claculate t values of direction to produce each of the voxel dimension sizes
    deltas = abs(np.divide(voxel_size,direction))

    point_modulo = ((point-bottom_left) / voxel_size) - np.floor((point-bottom_left) / voxel_size)
    point_modulo = (point-bottom_left) % voxel_size

    #find the maximum t vlues for the direction that can be moved before crossing a voxel

    # diag = ((direction < 0) * -2 + 1) * voxel_size
    # print(((direction < 0) * -2 + 1))
    # print(diag)

    # tmaxs = np.divide((diag-point_modulo), direction)

    tmaxs = direction * 0
    tmaxs[(direction < 0)] = point_modulo[(direction < 0)]
    tmaxs[(direction > 0)] = voxel_size[(direction > 0)] - point_modulo[(direction > 0)]

    # tmaxs[(point_modulo == 0) & (direction < 0)] = 0
    # tmaxs[(point_modulo == 0) & ((direction == 0)  & (point == top_right))] = 0
    

    #find the voxel coord of the voxel that contains the point
    vox_coord = np.floor((point-bottom_left)/ voxel_size)
    return vox_coord, steps, deltas, tmaxs

voxel_counts = np.array([2, 2])
bottom_left = np.array([-1,-1])
top_right = np.array([1,1])
voxel_size = (top_right - bottom_left)/voxel_counts

point = np.array([0,0])
direction = np.array([1,-1])
direction = direction/np.linalg.norm(direction)
print("pt = ", point, " dir = ", direction)
start_coord, steps, deltas, tmaxs = get_containing_voxel(bottom_left, top_right, voxel_size, point, direction)
print("start_coord ", start_coord)
print("steps ", steps)
print("deltas ", deltas)
print("tmaxs ", tmaxs)

pt =  [0 0]  dir =  [ 0.70710678 -0.70710678]
[ 1 -1]
[ 1. -1.]
start_coord  [1. 1.]
steps  [ 1 -1]
deltas  [1.41421356 1.41421356]
tmaxs  [1. 0.]


In [12]:
def trace_no_scaling(ray_origin, start_coord, tmaxs, deltas, voxel_counts):
    lowest = np.array([0,0,0])
    while True:
        print(start_coord)
        if tmaxs[0] < tmaxs[1]:
            if tmaxs[0] < tmaxs[2]:
                start_coord[0] = start_coord[0] + steps[0]
                tmaxs[0] = tmaxs[0] + deltas[0]
            else:
                start_coord[2] = start_coord[2] + steps[2]
                tmaxs[2] = tmaxs[2] + deltas[2]
        else:
            if tmaxs[1] < tmaxs[2]:
                start_coord[1] = start_coord[1] + steps[1]
                tmaxs[1] = tmaxs[1] + deltas[1]
            else:
                start_coord[2] = start_coord[2] + steps[2]
                tmaxs[2] = tmaxs[2] + deltas[2]
        print(tmaxs)
        if np.any(start_coord >= voxel_counts) or np.any(start_coord < lowest):
            break

In [13]:
def trace_scaling(start_coord, tmaxs, deltas, voxel_counts):
    out_of_range_count = 0
    lowest = np.array([0,0,0])
    print(start_coord)
    while True:
        dim = 0
        if tmaxs[0] < tmaxs[1]:
            if tmaxs[0] < tmaxs[2]:
                start_coord[0] = start_coord[0] + steps[0]
                dim = 0
            else:
                start_coord[2] = start_coord[2] + steps[2]
                dim = 2
        else:
            if tmaxs[1] < tmaxs[2]:
                start_coord[1] = start_coord[1] + steps[1]
                dim = 1
            else:
                start_coord[2] = start_coord[2] + steps[2]
                dim = 2
        if np.any(start_coord >= voxel_counts) or np.any(start_coord < lowest):
            print("ignore")
            out_of_range_count = out_of_range_count + 1
        if out_of_range_count == 4:
            break
        print("dim = ", dim, " t = ", tmaxs[dim])
        tmaxs = tmaxs - tmaxs[dim]
        tmaxs[dim] = deltas[dim]
        print(start_coord, tmaxs)
        print()
        

In [56]:
# xvoxels = 117
# yvoxels = 117
# zvoxels = 63
xvoxels = 4
yvoxels = 2
zvoxels = 2
voxel_counts = np.array([xvoxels, yvoxels, zvoxels])
bottom_left = np.array([-1,-1,-1])
top_right = np.array([1,1,1])
voxel_size = (top_right - bottom_left)/voxel_counts
print(voxel_size, voxel_counts)
print(voxel_size*voxel_counts)

ray_origin = np.array([1, 1, 1])
ray_direction = np.array([-1,-1,0])

# ray_origin = np.array([-0.94919843, -0.22988174,  1])
# ray_direction = np.array([-0.75137672, -0.6421716,  -0.15181786])

# ray_origin = np.array([0.98586982, 0.96225916, 1])
# ray_direction = np.array([0.19116649, 0.96221394, -0.19390643])

# ray_origin = ray_origin * -1
# ray_direction = ray_direction* -1
ray_direction = ray_direction/np.linalg.norm(ray_direction)

start_coord, steps, deltas, tmaxs = get_containing_voxel(bottom_left, top_right, voxel_size, ray_origin, ray_direction)
print("start_coord ", start_coord)
print("steps ", steps)
print("deltas ", deltas)
print("tmaxs ", tmaxs)

trace_scaling(start_coord, tmaxs, deltas, voxel_counts)



[0.5 1.  1. ] [4 2 2]
[2. 2. 2.]
[-1 -1  1]
[-0.5 -1.   1. ]


C:\Users\YannikNelson\AppData\Local\Temp\ipykernel_21888\3025072664.py:12: RuntimeWarning: divide by zero encountered in true_divide
  deltas = abs(np.divide(voxel_size,direction))


IndexError: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 3